In [1]:
%pip install ultralytics --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
from ultralytics.models.yolo.detect.val import DetectionValidator, check_requirements, LOGGER, Path

class BaseCustomDetectionValidator(DetectionValidator):
    def eval_json_orig(self, stats):
        """Evaluates YOLO output in JSON format and returns performance statistics."""
        if self.args.save_json and (self.is_coco or self.is_lvis) and len(self.jdict):
            pred_json = self.save_dir / "predictions.json"  # predictions
            anno_json = (
                self.data["path"]
                / "annotations"
                / ("instances_val2017.json" if self.is_coco else f"lvis_v1_{self.args.split}.json")
            )  # annotations
            pkg = "pycocotools" if self.is_coco else "lvis"
            LOGGER.info(f"\nEvaluating {pkg} mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                for x in pred_json, anno_json:
                    assert x.is_file(), f"{x} file not found"
                check_requirements("pycocotools>=2.0.6" if self.is_coco else "lvis>=0.5.3")
                if self.is_coco:
                    from pycocotools.coco import COCO  # noqa
                    from pycocotools.cocoeval import COCOeval  # noqa

                    anno = COCO(str(anno_json))  # init annotations api
                    pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                    val = COCOeval(anno, pred, "bbox")
                else:
                    from lvis import LVIS, LVISEval

                    anno = LVIS(str(anno_json))  # init annotations api
                    pred = anno._load_json(str(pred_json))  # init predictions api (must pass string, not Path)
                    val = LVISEval(anno, pred, "bbox")
                val.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # images to eval
                val.evaluate()
                val.accumulate()
                val.summarize()
                if self.is_lvis:
                    val.print_results()  # explicitly call print_results
                # update mAP50-95 and mAP50
                stats[self.metrics.keys[-1]], stats[self.metrics.keys[-2]] = (
                    val.stats[:2] if self.is_coco else [val.results["AP50"], val.results["AP"]]
                )
            except Exception as e:
                LOGGER.warning(f"{pkg} unable to run: {e}")
        return stats
    
    def eval_json_faster(self, stats):
        """Evaluates YOLO output in JSON format and returns performance statistics."""
        if self.args.save_json and (self.is_coco or self.is_lvis) and len(self.jdict):
            pred_json = self.save_dir / "predictions.json"  # predictions
            anno_json = (
                self.data["path"]
                / "annotations"
                / ("instances_val2017.json" if self.is_coco else f"lvis_v1_{self.args.split}.json")
            )  # annotations
            pkg = "faster_coco_eval"
            LOGGER.info(f"\nEvaluating {pkg} mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                for x in pred_json, anno_json:
                    assert x.is_file(), f"{x} file not found"
                
                from faster_coco_eval import COCO, COCOeval_faster

                if self.is_coco:
                    anno = COCO(str(anno_json))  # init annotations api
                    pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                    val = COCOeval_faster(anno, pred, "bbox", print_function=print)
                else:
                    anno = COCO(str(anno_json))  # init annotations api
                    pred = anno._load_json(str(pred_json))  # init predictions api (must pass string, not Path)
                    val = COCOeval_faster(anno, pred, "bbox", lvis_style=True, print_function=print)
                    val.params.maxDets = [300]
                
                val.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # images to eval
                val.evaluate()
                val.accumulate()
                val.summarize()
                
                # update mAP50-95 and mAP50
                stats[self.metrics.keys[-1]], stats[self.metrics.keys[-2]] = val.stats[:2]
            except Exception as e:
                LOGGER.warning(f"{pkg} unable to run: {e}")
        return stats
    
    def eval_json(self, stats):
        tic_faster = time.time()
        self.eval_json_faster(stats)
        toc_faster = time.time()
        print(f"Faster eval took {toc_faster - tic_faster:.2f}s")

        
        tic_orig = time.time()
        stats = self.eval_json_orig(stats)
        toc_orig = time.time()
        print(f"Original eval took {toc_orig - tic_orig:.2f}s")
        
        return stats

In [3]:
!rm -rf rm -rf runs/

args = dict(model='yolov8n.pt', data='./coco_val_only.yaml')
validator = BaseCustomDetectionValidator(args=args)
validator()

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco_val_only.yaml' images not found ⚠️, missing path '/home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco/val2017.txt'


/home/mixaill76/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 169M/169M [00:06<00:00, 25.8MB/s] 
Unzipping /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco2017labels-segments.zip to /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco...: 100%|██████████| 122232/122232 [00:07<00:00, 16920.33file/s]


Dataset download success ✅ (100.7s), saved to /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets



val: Scanning /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco/labels/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:05<00:00, 864.61it/s]


val: New cache created: /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco/labels/val2017.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:31<00:00,  9.86it/s]


                   all       5000      36335      0.629      0.476      0.521       0.37
                person       5000      10777      0.751      0.678      0.745      0.515
               bicycle       5000        314      0.694      0.411      0.466      0.269
                   car       5000       1918      0.656      0.527      0.566      0.364
            motorcycle       5000        367       0.71      0.573      0.654      0.412
              airplane       5000        143      0.755      0.776      0.845      0.654
                   bus       5000        283       0.73      0.664      0.739      0.621
                 train       5000        190      0.795      0.774      0.833      0.648
                 truck       5000        414      0.519      0.384       0.45      0.301
                  boat       5000        424      0.562      0.297      0.373      0.209
         traffic light       5000        634      0.641      0.352      0.415      0.213
          fire hydran

{'metrics/precision(B)': 0.6292564172491455,
 'metrics/recall(B)': 0.47631066232459646,
 'metrics/mAP50(B)': 0.5255884728465783,
 'metrics/mAP50-95(B)': 0.3731985247705999,
 'fitness': 0.38551831525427444}

### Faster eval took 7.34s

### Original eval took 34.35s

In [1]:
import time
from ultralytics.models.yolo.segment.val import SegmentationValidator, check_requirements, LOGGER, Path

class BaseCustomSegmentationValidator(SegmentationValidator):
    def eval_json_orig(self, stats):
        """Return COCO-style object detection evaluation metrics."""
        if self.args.save_json and self.is_coco and len(self.jdict):
            anno_json = self.data["path"] / "annotations/instances_val2017.json"  # annotations
            pred_json = self.save_dir / "predictions.json"  # predictions
            LOGGER.info(f"\nEvaluating pycocotools mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                check_requirements("pycocotools>=2.0.6")
                from pycocotools.coco import COCO  # noqa
                from pycocotools.cocoeval import COCOeval  # noqa

                for x in anno_json, pred_json:
                    assert x.is_file(), f"{x} file not found"
                anno = COCO(str(anno_json))  # init annotations api
                pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                for i, eval in enumerate([COCOeval(anno, pred, "bbox"), COCOeval(anno, pred, "segm")]):
                    if self.is_coco:
                        eval.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # im to eval
                    eval.evaluate()
                    eval.accumulate()
                    eval.summarize()
                    idx = i * 4 + 2
                    stats[self.metrics.keys[idx + 1]], stats[self.metrics.keys[idx]] = eval.stats[
                        :2
                    ]  # update mAP50-95 and mAP50
            except Exception as e:
                LOGGER.warning(f"pycocotools unable to run: {e}")
        return stats
    
    def eval_json_faster(self, stats):
        """Return COCO-style object detection evaluation metrics."""
        if self.args.save_json and self.is_coco and len(self.jdict):
            anno_json = self.data["path"] / "annotations/instances_val2017.json"  # annotations
            pred_json = self.save_dir / "predictions.json"  # predictions
            LOGGER.info(f"\nEvaluating pycocotools mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                from faster_coco_eval import COCO, COCOeval_faster

                for x in anno_json, pred_json:
                    assert x.is_file(), f"{x} file not found"
                anno = COCO(str(anno_json))  # init annotations api
                pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                for i, eval in enumerate([COCOeval_faster(anno, pred, "bbox", print_function=print), COCOeval_faster(anno, pred, "segm", print_function=print)]):
                    if self.is_coco:
                        eval.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # im to eval
                    eval.evaluate()
                    eval.accumulate()
                    eval.summarize()
                    idx = i * 4 + 2
                    stats[self.metrics.keys[idx + 1]], stats[self.metrics.keys[idx]] = eval.stats[
                        :2
                    ]  # update mAP50-95 and mAP50
            except Exception as e:
                LOGGER.warning(f"faster_coco_eval unable to run: {e}")
        return stats

    def eval_json(self, stats):
        tic_faster = time.time()
        self.eval_json_faster(stats)
        toc_faster = time.time()
        print(f"Faster eval took {toc_faster - tic_faster:.2f}s")

        
        tic_orig = time.time()
        stats = self.eval_json_orig(stats)
        toc_orig = time.time()
        print(f"Original eval took {toc_orig - tic_orig:.2f}s")
        
        return stats

In [2]:
!rm -rf rm -rf runs/

args = dict(model='yolov8n-seg.pt', data='./coco_val_only.yaml')
validator = BaseCustomSegmentationValidator(args=args)
validator()

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)


/home/mixaill76/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs


val: Scanning /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [10:36<00:00,  2.03s/it]


                   all       5000      36335       0.62       0.48      0.516      0.363      0.617      0.456      0.489      0.305
                person       5000      10777      0.742       0.68      0.743      0.514       0.74      0.654      0.711        0.4
               bicycle       5000        314      0.666      0.366      0.438      0.243      0.579      0.315      0.346      0.132
                   car       5000       1918      0.616      0.508      0.556      0.356      0.626      0.486      0.531      0.296
            motorcycle       5000        367      0.709      0.569      0.668      0.405      0.686      0.523      0.589      0.287
              airplane       5000        143      0.722       0.78       0.83      0.643      0.725      0.762        0.8      0.493
                   bus       5000        283      0.768        0.7      0.749      0.612      0.762      0.679      0.731      0.556
                 train       5000        190      0.791      0.775   

{'metrics/precision(B)': 0.6199420382660138,
 'metrics/recall(B)': 0.47978746163511454,
 'metrics/mAP50(B)': 0.5207700497188987,
 'metrics/mAP50-95(B)': 0.36648003262764034,
 'metrics/precision(M)': 0.6168869715016637,
 'metrics/recall(M)': 0.45590238258381943,
 'metrics/mAP50(M)': 0.4896069455217768,
 'metrics/mAP50-95(M)': 0.30289961138089894,
 'fitness': 0.7018358719301855}

### Faster eval took 26.51s

### Original eval took 86.79s

In [1]:
import time
from ultralytics.models.yolo.pose.val import PoseValidator, check_requirements, LOGGER, Path

class BaseCustomPoseValidator(PoseValidator):
    def eval_json_orig(self, stats):
        """Evaluates object detection model using COCO JSON format."""
        if self.args.save_json and self.is_coco and len(self.jdict):
            anno_json = self.data["path"] / "annotations/person_keypoints_val2017.json"  # annotations
            pred_json = self.save_dir / "predictions.json"  # predictions
            LOGGER.info(f"\nEvaluating pycocotools mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                check_requirements("pycocotools>=2.0.6")
                from pycocotools.coco import COCO  # noqa
                from pycocotools.cocoeval import COCOeval  # noqa

                for x in anno_json, pred_json:
                    assert x.is_file(), f"{x} file not found"
                anno = COCO(str(anno_json))  # init annotations api
                pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                for i, eval in enumerate([COCOeval(anno, pred, "bbox"), COCOeval(anno, pred, "keypoints")]):
                    if self.is_coco:
                        eval.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # im to eval
                    eval.evaluate()
                    eval.accumulate()
                    eval.summarize()
                    idx = i * 4 + 2
                    stats[self.metrics.keys[idx + 1]], stats[self.metrics.keys[idx]] = eval.stats[
                        :2
                    ]  # update mAP50-95 and mAP50
            except Exception as e:
                LOGGER.warning(f"pycocotools unable to run: {e}")
        return stats
    
    def eval_json_faster(self, stats):
        """Evaluates object detection model using COCO JSON format."""
        if self.args.save_json and self.is_coco and len(self.jdict):
            anno_json = self.data["path"] / "annotations/person_keypoints_val2017.json"  # annotations
            pred_json = self.save_dir / "predictions.json"  # predictions
            LOGGER.info(f"\nEvaluating pycocotools mAP using {pred_json} and {anno_json}...")
            try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
                from faster_coco_eval import COCO, COCOeval_faster

                for x in anno_json, pred_json:
                    assert x.is_file(), f"{x} file not found"
                anno = COCO(str(anno_json))  # init annotations api
                pred = anno.loadRes(str(pred_json))  # init predictions api (must pass string, not Path)
                for i, eval in enumerate([COCOeval_faster(anno, pred, "bbox", print_function=print), COCOeval_faster(anno, pred, "keypoints", print_function=print)]):
                    if self.is_coco:
                        eval.params.imgIds = [int(Path(x).stem) for x in self.dataloader.dataset.im_files]  # im to eval
                    eval.evaluate()
                    eval.accumulate()
                    eval.summarize()
                    idx = i * 4 + 2
                    stats[self.metrics.keys[idx + 1]], stats[self.metrics.keys[idx]] = eval.stats[
                        :2
                    ]  # update mAP50-95 and mAP50
            except Exception as e:
                LOGGER.warning(f"pycocotools unable to run: {e}")
        return stats
    
    def eval_json(self, stats):
        tic_faster = time.time()
        self.eval_json_faster(stats)
        toc_faster = time.time()
        print(f"Faster eval took {toc_faster - tic_faster:.2f}s")

        
        tic_orig = time.time()
        stats = self.eval_json_orig(stats)
        toc_orig = time.time()
        print(f"Original eval took {toc_orig - tic_orig:.2f}s")
        
        return stats

In [2]:
!rm -rf rm -rf runs/

args = dict(model='yolov8n-pose.pt', data='./coco_pose_val_only.yaml')
validator = BaseCustomPoseValidator(args=args)
validator()

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv8n-pose summary (fused): 187 layers, 3289964 parameters, 0 gradients, 9.2 GFLOPs


/home/mixaill76/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
val: Scanning /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco-pose/labels/val2017.cache... 2346 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2346/2346 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:17<00:00,  8.18it/s]


                   all       2346       6352      0.867      0.821      0.909      0.694      0.834      0.749      0.798      0.509
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Saving runs/detect/train/predictions.json...

Evaluating pycocotools mAP using runs/detect/train/predictions.json and /home/mixaill76/faster_coco_eval/examples/ultralytics/datasets/coco-pose/annotations/person_keypoints_val2017.json...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished...
DONE (t=0.36s).
Accumulating evaluation results...
COCOeval_opt.accumulate() finished...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.526
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.713
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.584
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium 

{'metrics/precision(B)': 0.8673525355617393,
 'metrics/recall(B)': 0.820528967254408,
 'metrics/mAP50(B)': 0.7133781833288113,
 'metrics/mAP50-95(B)': 0.5257107235960686,
 'metrics/precision(P)': 0.8337325944732837,
 'metrics/recall(P)': 0.7485831234256927,
 'metrics/mAP50(P)': 0.799166115657517,
 'metrics/mAP50-95(P)': 0.5049564874508521,
 'fitness': 1.2539324350911183}

### Faster eval took 4.45s

### Original eval took 10.17s

### ultralytics eval compare considering data loading

| lib | model | time | profit | 
| ---- | ----- | ---- | ------ |
| faster-coco-eval  | bbox | 7.34 | 4.5x |
| pycocotools  | bbox | 34.35 | 1x |
| ---- | ----- | ---- | ------ |
| faster-coco-eval  | bbox+segm | 26.51 | 3.2x |
| pycocotools  | bbox+segm | 86.79 | 1x |
| ---- | ----- | ---- | ------ |
| faster-coco-eval  | bbox+keypoints | 4.45 | 2,2x |
| pycocotools  | bbox+keypoints | 10.17 | 1x |

